In [35]:
import os
import pandas as pd
import plotly
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import plotly as pys
import ipywidgets as widgets
from ipywidgets import Layout

import _global_scripts as gs

## Purpose

E2.13 - Investigation #13: Implement dCRT ASC for distance bins
- E2.13.1 - Single TLF, RUNFACTOR = 1.5
- E2.13.2 - Single TLF, RUNFACTOR = 2.5
- E2.13.3 - 


## Inputs

In [36]:
# set whether you want PA or OD analysis
pa_od = 'PA'
pa_od_function = 'pa'

In [37]:
df_tdm_obs = pd.read_csv(f"_data/base_observed_summary_{pa_od_function}.csv").sort_values(by=['Source','station'], ascending=True).reset_index(drop=True)
if(pa_od == 'PA'):
        df_tdm_obs = df_tdm_obs.drop(columns={'Alt_PA','Alt_Direct_PA','Alt_Transfer_PA'})

# read in results for e2.13.1 before recalibration
path_brding_summary_2131 = r"_data/E2.13/E2.13.1_node.csv"
path_rider_summary_2131  = r"_data/E2.13/E2.13.1_link.csv"
path_brding_summary_2132 = r"_data/E2.13/E2.13.2_node.csv"
path_rider_summary_2132  = r"_data/E2.13/E2.13.2_link.csv"

In [ ]:
# read in stations and summarize tdm results
df_stations1 = gs.df_stations[['station','N']]
df_tdm_2131 = gs.summarize_tdm_stats(path_brding_summary_2131,path_rider_summary_2131, df_stations1, 'TDM_2.13.1', pa_od_function)
df_tdm_2132 = gs.summarize_tdm_stats(path_brding_summary_2132,path_rider_summary_2132, df_stations1, 'TDM_2.13.2', pa_od_function)


# concat dataframes
df_tdm_obs_new = pd.concat([
        df_tdm_obs,
        df_tdm_2131,
        df_tdm_2132
    ]).reset_index()
df_tdm_obs_new = df_tdm_obs_new.round(3).fillna(0)
df_tdm_obs_new = df_tdm_obs_new.drop(columns='index')
df_tdm_obs_new = df_tdm_obs_new.loc[:, ~df_tdm_obs_new.columns.str.contains('^Unnamed')]

# rename
sumStats = df_tdm_obs_new.copy()
display(df_tdm_obs_new)

e:\GitHub\WF-TDM-v9x\_scripts\_global_scripts.py:27: DtypeWarning:

Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.

e:\GitHub\WF-TDM-v9x\_scripts\_global_scripts.py:27: DtypeWarning:

Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.



,Source,station,AccessMode,Brd_PA,Brd_Direct_PA,Brd_Transfer_PA
0,OBS,01-PROVO CENTRAL STATION,drive,2283.379,0.00,0.00
1,OBS,01-PROVO CENTRAL STATION,walk,945.908,0.00,0.00
2,OBS,02-OREM CENTRAL STATION,drive,1475.024,0.00,0.00
3,OBS,02-OREM CENTRAL STATION,walk,371.804,0.00,0.00
4,OBS,03-AMERICAN FORK STATION,drive,1423.768,0.00,0.00
...,...,...,...,...,...,...
115,TDM_2.13.1,11-FARMINGTON STATION,walk,196.720,135.59,61.13
116,TDM_2.13.1,12-LAYTON STATION,walk,362.360,140.23,222.13
117,TDM_2.13.1,13-CLEARFIELD STATION,walk,633.370,504.98,128.39
118,TDM_2.13.1,14-ROY STATION,walk,386.180,349.84,36.34


In [39]:
# add a few more columns regarding percentage of boardings in relation to total boardings

# sum by source and station 
station_sum = sumStats.groupby(["Source", "station"], as_index=False).agg({
    f"Brd_{pa_od}": "sum",
    f"Brd_Direct_{pa_od}": "sum",
    f"Brd_Transfer_{pa_od}": "sum"
})

# add All accessMode
station_sum["AccessMode"] = "All"
sumStats2 = pd.concat([sumStats, station_sum], ignore_index=True) 

accessmode_sum = sumStats2.groupby(["Source", "AccessMode"], as_index=False).agg({
    f"Brd_{pa_od}": "sum",
    f"Brd_Direct_{pa_od}": "sum",
    f"Brd_Transfer_{pa_od}": "sum"
})

accessmode_sum.rename(columns={
    f"Brd_{pa_od}": f"Source_Brd_{pa_od}", 
    f"Brd_Direct_{pa_od}": f"Source_Brd_Direct_{pa_od}", 
    f"Brd_Transfer_{pa_od}": f"Source_Brd_Transfer_{pa_od}"}, inplace=True)
sumStatsP = sumStats2.merge(accessmode_sum, on=["Source", "AccessMode"], how="left")

sumStatsP[f"Brd_{pa_od}_Perc"]          = sumStatsP[f"Brd_{pa_od}"] / sumStatsP[f"Source_Brd_{pa_od}"]
sumStatsP[f"Brd_Direct_{pa_od}_Perc"]   = sumStatsP[f"Brd_Direct_{pa_od}"] / sumStatsP[f"Source_Brd_Direct_{pa_od}"]
sumStatsP[f"Brd_Transfer_{pa_od}_Perc"] = sumStatsP[f"Brd_Transfer_{pa_od}"] / sumStatsP[f"Source_Brd_Transfer_{pa_od}"]
sumStatsP


,Source,station,AccessMode,Brd_PA,Brd_Direct_PA,Brd_Transfer_PA,Source_Brd_PA,Source_Brd_Direct_PA,Source_Brd_Transfer_PA,Brd_PA_Perc,Brd_Direct_PA_Perc,Brd_Transfer_PA_Perc
0,OBS,01-PROVO CENTRAL STATION,drive,2283.379,0.00,0.00,14955.118,0.00,0.00,0.152682,NaN,NaN
1,OBS,01-PROVO CENTRAL STATION,walk,945.908,0.00,0.00,5672.892,0.00,0.00,0.166742,NaN,NaN
2,OBS,02-OREM CENTRAL STATION,drive,1475.024,0.00,0.00,14955.118,0.00,0.00,0.098630,NaN,NaN
3,OBS,02-OREM CENTRAL STATION,walk,371.804,0.00,0.00,5672.892,0.00,0.00,0.065540,NaN,NaN
4,OBS,03-AMERICAN FORK STATION,drive,1423.768,0.00,0.00,14955.118,0.00,0.00,0.095203,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
160,TDM_2.13.1,11-FARMINGTON STATION,All,1285.230,1166.00,119.23,39691.420,32369.91,7321.51,0.032381,0.036021,0.016285
161,TDM_2.13.1,12-LAYTON STATION,All,3518.660,3068.93,449.73,39691.420,32369.91,7321.51,0.088650,0.094808,0.061426
162,TDM_2.13.1,13-CLEARFIELD STATION,All,3259.450,3006.37,253.08,39691.420,32369.91,7321.51,0.082120,0.092875,0.034567
163,TDM_2.13.1,14-ROY STATION,All,3194.240,3121.95,72.29,39691.420,32369.91,7321.51,0.080477,0.096446,0.009874


In [40]:
def plotit(variable, access_mode):
    output.clear_output()  # Clear previous output before displaying new content
    global firstTime
    if firstTime:
    
        filtered_data = sumStatsP[sumStatsP['AccessMode'] == access_mode]
            
        # Create histogram
        fig = px.histogram(
            filtered_data, 
            x="station", 
            y=variable, 
            text_auto='.2s',
            color='Source', 
            barmode='group',
            height=400
        )
        fig.update_layout(
            xaxis_title="Station Name",
            yaxis_title=str(variable),
            legend_title="Model Version"
        )
        
        # Display the plot
        fig.show()
    
    else:
        firstTime = True

In [41]:
lstValues = list([
    f'Brd_{pa_od}',
    f'Brd_Direct_{pa_od}',
    f'Brd_Transfer_{pa_od}',
    f'Brd_{pa_od}_Perc',
    f'Brd_Direct_{pa_od}_Perc',
    f'Brd_Transfer_{pa_od}_Perc'])
accessModeOptions = ['drive', 'walk', 'All']

selectValues = widgets.Select(options=lstValues, value=(f'Brd_{pa_od}' ), description = 'Select Variable')
selectAccessMode = widgets.Dropdown(options=accessModeOptions, value='All', description='Access Mode')

# Set up a global variable to track whether the widgets have been changed
firstTime = False

# create output widget to display filtered DataFrame
output = widgets.Output()
hbox = widgets.HBox([selectValues, selectAccessMode])

# create interactive widget
interactive_output = widgets.interactive_output(plotit, {'variable':selectValues, 'access_mode': selectAccessMode})

display(hbox)
display(interactive_output)
display(output)

Output()

Output()

## Trip Length Distribution

In [42]:
# Files to copy to _large-files\WF-TDM-v9x-v920-E2\E2.12

#  2_DetailedTripMatrices\AllTrips_pkok.omx
#  2_DetailedTripMatrices\AllTrips_Pk.omx
#  2_DetailedTripMatrices\AllTrips_Ok.omx
#  1a_Skims\skm_d8_Pk.omx
#  1a_Skims\skm_d8_Ok.omx
#  1a_Skims\skm_w8_Pk.omx
#  1a_Skims\skm_w8_Ok.omx

In [43]:
sumStatsP

,Source,station,AccessMode,Brd_PA,Brd_Direct_PA,Brd_Transfer_PA,Source_Brd_PA,Source_Brd_Direct_PA,Source_Brd_Transfer_PA,Brd_PA_Perc,Brd_Direct_PA_Perc,Brd_Transfer_PA_Perc
0,OBS,01-PROVO CENTRAL STATION,drive,2283.379,0.00,0.00,14955.118,0.00,0.00,0.152682,NaN,NaN
1,OBS,01-PROVO CENTRAL STATION,walk,945.908,0.00,0.00,5672.892,0.00,0.00,0.166742,NaN,NaN
2,OBS,02-OREM CENTRAL STATION,drive,1475.024,0.00,0.00,14955.118,0.00,0.00,0.098630,NaN,NaN
3,OBS,02-OREM CENTRAL STATION,walk,371.804,0.00,0.00,5672.892,0.00,0.00,0.065540,NaN,NaN
4,OBS,03-AMERICAN FORK STATION,drive,1423.768,0.00,0.00,14955.118,0.00,0.00,0.095203,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
160,TDM_2.13.1,11-FARMINGTON STATION,All,1285.230,1166.00,119.23,39691.420,32369.91,7321.51,0.032381,0.036021,0.016285
161,TDM_2.13.1,12-LAYTON STATION,All,3518.660,3068.93,449.73,39691.420,32369.91,7321.51,0.088650,0.094808,0.061426
162,TDM_2.13.1,13-CLEARFIELD STATION,All,3259.450,3006.37,253.08,39691.420,32369.91,7321.51,0.082120,0.092875,0.034567
163,TDM_2.13.1,14-ROY STATION,All,3194.240,3121.95,72.29,39691.420,32369.91,7321.51,0.080477,0.096446,0.009874


In [44]:
sumStatsPg = sumStatsP.groupby(['Source','AccessMode'],as_index=False).agg(Brd_PA=('Brd_PA','sum'))
sumStatsPg

,Source,AccessMode,Brd_PA
0,OBS,All,20628.010
1,OBS,drive,14955.118
2,OBS,walk,5672.892
3,TDM,All,18575.280
4,TDM,drive,12159.620
5,TDM,walk,6415.660
6,TDM_2.13.1,All,39691.420
7,TDM_2.13.1,drive,26817.470
8,TDM_2.13.1,walk,12873.950


In [45]:

# Pivot to make AccessMode columns
pivot_df = sumStatsPg.pivot(index='Source', columns='AccessMode', values='Brd_PA').round(0)
pivot_df

AccessMode,All,drive,walk
Source,,,
OBS,20628.0,14955.0,5673.0
TDM,18575.0,12160.0,6416.0
TDM_2.13.1,39691.0,26817.0,12874.0
